<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/arbitraj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 10 adet coin'in fiyat verilerini çekme
def get_data(symbols, start_date, end_date):
    data = yf.download(symbols, start=start_date, end=end_date)['Adj Close']
    return data

# Z-score hesaplama
def calculate_zscore(series):
    return (series - series.mean()) / series.std()

# PCA tabanlı istatistiksel arbitraj stratejisi
def statistical_arbitrage(symbols, start_date, end_date):
    # Verileri çek
    data = get_data(symbols, start_date, end_date)
    data = data.dropna()

    # Log getirileri hesapla
    log_returns = np.log(data / data.shift(1)).dropna()

    # Verileri standardize et
    scaler = StandardScaler()
    scaled_returns = scaler.fit_transform(log_returns)

    # PCA uygula
    pca = PCA(n_components=2)  # İlk 2 bileşeni kullan
    pca.fit(scaled_returns)
    components = pca.transform(scaled_returns)

    # İlk bileşeni spread olarak kullan
    spread = components[:, 0]

    # Z-score hesapla
    zscore = calculate_zscore(spread)

    # Alım/Satım sinyalleri
    threshold = 1.0
    data['position'] = np.where(zscore > threshold, -1, np.nan)
    data['position'] = np.where(zscore < -threshold, 1, data['position'])
    data['position'] = np.where(np.abs(zscore) < 0.5, 0, data['position'])
    data['position'] = data['position'].ffill().fillna(0)

    # Getiri hesapla
    data['strategy_return'] = data['position'].shift(1) * log_returns.mean(axis=1)
    data['cumulative_return'] = (data['strategy_return'] + 1).cumprod()

    # Sonuçları görselleştir
    plt.figure(figsize=(14, 7))
    plt.plot(data['cumulative_return'], label='Strateji Getirisi')
    plt.title('10 Coin İstatistiksel Arbitraj Stratejisi')
    plt.legend()
    plt.show()

    return data

# Örnek kullanım
symbols = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'ADA-USD', 'DOGE-USD',
           'XRP-USD', 'DOT-USD', 'UNI-USD', 'LTC-USD', 'LINK-USD']
start_date = '2020-01-01'
end_date = '2023-01-01'

results = statistical_arbitrage(symbols, start_date, end_date)